In [ ]:
import os   
print("Train classes:", os.listdir("dateset/train"))
print("Test classes :", os.listdir("dateset/test"))

Train classes: ['அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'ஔ']
Test classes : ['அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'ஔ']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import json
# -----------------------------
# PARAMETERS
# -----------------------------
IMG_SIZE = 128  # Increased from 64 for better accuracy
BATCH_SIZE = 16
EPOCHS = 100  # Increased from 30
NUM_CLASSES = 12

TRAIN_DIR = "dateset/train"
TEST_DIR  = "dateset/test"

# -----------------------------
# DATA GENERATORS WITH AUGMENTATION
# -----------------------------
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
test_gen  = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# -----------------------------
# IMPROVED CNN MODEL WITH MORE DEPTH & REGULARIZATION
# -----------------------------
model = Sequential([
    # Block 1
    Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'),
    BatchNormalization(),
    Conv2D(32, (3,3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),
    
    # Block 2
    Conv2D(64, (3,3), activation="relu", padding='same'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),
    
    # Block 3
    Conv2D(128, (3,3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),
    
    # Fully Connected
    Flatten(),
    Dense(128, activation="relu"),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# -----------------------------
# TRAIN WITH EARLY STOPPING
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=[early_stop]
)

# -----------------------------
# SAVE MODEL
# -----------------------------
model.save("uyir_cnn_model.h5")
print("✅ Model saved as uyir_cnn_model.h5")

# -----------------------------
# SAVE CLASS → LETTER MAPPING
# -----------------------------
class_indices = train_data.class_indices
index_to_letter = {v: k for k, v in class_indices.items()}

with open("uyir_label_map.json", "w", encoding="utf-8") as f:
    json.dump(index_to_letter, f, ensure_ascii=False, indent=2)

print("✅ Label mapping saved as uyir_label_map.json")
print("Label mapping:", index_to_letter)

Found 970 images belonging to 12 classes.
Found 239 images belonging to 12 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 822,476 (3.14 MB)

 Trainable params: 822,476 (3.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.1660 - loss: 2.4084 - val_accuracy: 0.3096 - val_loss: 2.1305
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.4340 - loss: 1.7805 - val_accuracy: 0.4142 - val_loss: 1.8796
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.5680 - loss: 1.3026 - val_accuracy: 0.4686 - val_loss: 1.7551
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.6866 - loss: 0.9732 - val_accuracy: 0.5356 - val_loss: 1.5462
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.7804 - loss: 0.7227 - val_accuracy: 0.5607 - val_loss: 1.6014
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8351 - loss: 0.5523 - val_accuracy: 0.6025 - val_loss: 1.2862
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8835 - loss: 0.3810 - val_accuracy: 0.6360 - val_loss: 1.2487
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9309 - loss: 0.2427 - val_accuracy: 0.6360 - v

✅ Model saved as uyir_cnn_model.h5
✅ Label mapping saved as uyir_label_map.json
Label mapping: {0: 'அ', 1: 'ஆ', 2: 'இ', 3: 'ஈ', 4: 'உ', 5: 'ஊ', 6: 'எ', 7: 'ஏ', 8: 'ஐ', 9: 'ஒ', 10: 'ஓ', 11: 'ஔ'}


In [ ]:
import cv2
import numpy as np
import json
from tensorflow.keras.models import load_model

# Load model
model = load_model("uyir_cnn_model.h5")

# Load label map
with open("uyir_label_map.json", "r", encoding="utf-8") as f:
    index_to_letter = json.load(f)

# Load image
img = cv2.imread("sample.jpeg", cv2.IMREAD_GRAYSCALE)

if img is None:
    print("❌ Image not found")
    exit()

# 🔴 IMPORTANT: MUST MATCH TRAINING SIZE
img = cv2.resize(img, (64, 64))   # <-- FIXED
img = img.astype("float32") / 255.0
img = img.reshape(1, 64, 64, 1)

# Predict
pred = model.predict(img)
pred_index = int(np.argmax(pred))
pred_letter = index_to_letter[str(pred_index)]

print("✅ Predicted letter:", pred_letter)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
✅ Predicted letter: ஔ


In [ ]:
# Load model and test data
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = load_model("uyir_cnn_model.h5")

# Load test data
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(
    "dateset/test",
    target_size=(128, 128),  # Updated to match new IMG_SIZE
    color_mode="grayscale",
    batch_size=16,
    class_mode="categorical",
    shuffle=False
)

# Evaluate model
loss, accuracy = model.evaluate(test_data)

print(f"✅ Test Loss: {loss:.4f}")
print(f"✅ Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

Found 239 images belonging to 12 classes.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3682 - loss: 1.7526
✅ Test Loss: 1.7526
✅ Test Accuracy: 0.3682 (36.82%)
